In [ ]:
import pandas as pd
import os

csv_path = "train_smote.csv" 
png_base_path = "/content/drive/MyDrive/dl_package/mri_png"    
print(f"CSV exists: {os.path.exists(csv_path)}")
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"CSV rows: {len(df)}")
    print(f"Columns: {list(df.columns)}")
    print(f"Sample data:\n{df.head()}")
    print(f"Label counts:\n{df['label'].value_counts()}")
print(f"\nPNG dir exists: {os.path.exists(png_base_path)}")
if os.path.exists(png_base_path):
    png_files = os.listdir(png_base_path)
    print(f"PNG files: {len(png_files)}")
    print(f"Sample PNGs: {png_files[:5]}")

CSV exists: True
CSV rows: 725
Columns: ['Image Data ID', 'Subject', 'mri_path', 'clinical', 'label']
Sample data:
           Image Data ID     Subject             mri_path  \
0                 I40445  032_S_0214  mri_png\I119084.png   
1                 I54651  073_S_0518  mri_png\I119233.png   
2                 I59711  003_S_0907   mri_png\I52781.png   
3                I119135  116_S_0392   mri_png\I59690.png   
4  synthetic_1187_I35841  099_S_0090   mri_png\I35841.png   

                         clinical  label  
0  [0.1714285714285714, 0.0, 1.0]      1  
1  [0.3714285714285714, 1.0, 0.0]      1  
2  [0.4857142857142855, 0.0, 1.0]      2  
3                 [0.8, 1.0, 0.0]      0  
4  [0.5714285714285714, 0.0, 1.0]      2  
Label counts:
label
2    242
0    242
1    241
Name: count, dtype: int64

PNG dir exists: True
PNG files: 830
Sample PNGs: ['I119175.png', 'I119180.png', 'I119300.png', 'I138912.png', 'I118779.png']


EfficientNet-B0 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, F1Score, AUROC
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm
import copy
from sklearn.metrics import classification_report
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os

class MultimodalEfficientNet(nn.Module):
    def __init__(self, num_classes=3, clinical_dim=3):
        super(MultimodalEfficientNet, self).__init__()
        self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True, in_chans=1)
        # Freezes conv_stem and bn1 (early layers) to preserve pretrained features
        # Freezing the stem retains generic features, allowing later layers to adapt to MRIs
        for param in self.efficientnet.conv_stem.parameters():
            param.requires_grad = False
        for param in self.efficientnet.bn1.parameters():
            param.requires_grad = False
        self.efficientnet.classifier = nn.Identity()
        self.feature_dim = 1280

        self.clinical_mlp = nn.Sequential(
            nn.Linear(clinical_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64),
            nn.ReLU()
        )

        self.fc = nn.Sequential(
            nn.Linear(1280 + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, images, clinical):
        img_features = self.efficientnet(images)
        clin_features = self.clinical_mlp(clinical)
        combined = torch.cat((img_features, clin_features), dim=1)
        output = self.fc(combined)
        return output

ADNIDataset class, transformations, and dataset loading

In [ ]:
class ADNIDataset(Dataset):
    def __init__(self, data_df, base_path="/content/drive/MyDrive/dl_package/mri_png", transform=None):
        self.data_df = data_df
        self.base_path = base_path
        self.transform = transform
        if len(self.data_df) == 0:
            raise ValueError(f"Dataset is empty: {len(self.data_df)} rows")

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        rel_path = self.data_df.iloc[idx]['mri_path'].replace('\\', '/').split('/')[-1]
        img_path = os.path.join(self.base_path, rel_path)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (224, 224))
        img = img.astype(np.float32) / 255.0
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        else:
            img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)
        clinical_str = self.data_df.iloc[idx]['clinical']
        clinical = np.array(eval(clinical_str), dtype=np.float32)
        clinical = torch.tensor(clinical, dtype=torch.float32)
        label = self.data_df.iloc[idx]['label']
        label = torch.tensor(label, dtype=torch.long)
        return img, clinical, label

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.6),
    A.RandomBrightnessContrast(p=0.4),
    A.Affine(scale=(0.9, 1.1), p=0.3),  
    A.Normalize(mean=[0.0], std=[1.0]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Normalize(mean=[0.0], std=[1.0]),
    ToTensorV2()
])

train_df = pd.read_csv("train_smote.csv")
val_df = pd.read_csv("val_smote.csv")
test_df = pd.read_csv("test_smote.csv")
print(f"Train samples: {len(train_df)}")
print(f"Val samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")
train_dataset = ADNIDataset(train_df, transform=train_transform)
val_dataset = ADNIDataset(val_df, transform=val_transform)
test_dataset = ADNIDataset(test_df, transform=val_transform)

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)


TRAINING PIPELINE

In [ ]:
model = MultimodalEfficientNet(num_classes=3, clinical_dim=3).to(device)

class_counts = train_df['label'].value_counts().sort_index()
class_weights = torch.tensor([1.0 / class_counts[i] for i in range(3)], dtype=torch.float32).to(device)
class_weights = class_weights / class_weights.sum() * 3
print("Class weights:", class_weights)
#MCI is harder to classify due to subtle MRI features
# modifies the standard cross-entropy (CE) loss by focusing training on difficult examples and reducing the influence of easy ones
def focal_loss(pred, target, gamma=3.0, alpha=class_weights):
    ce_loss = nn.functional.cross_entropy(pred, target, reduction='none')
    pt = torch.exp(-ce_loss)
    focal_loss = alpha[target] * (1 - pt) ** gamma * ce_loss
    return focal_loss.mean()

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=1e-4)
accuracy = Accuracy(task="multiclass", num_classes=3).to(device)
f1_score = F1Score(task="multiclass", num_classes=3, average="macro").to(device)
auroc = AUROC(task="multiclass", num_classes=3, average="macro").to(device)

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, num_epochs=50, patience=10):
    best_f1 = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    patience_counter = 0
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5)

    for epoch in range(num_epochs):
        model.train()
        train_loss, train_acc, train_f1, train_auc = 0.0, 0.0, 0.0, 0.0
        train_samples = 0

        print(f"\nEpoch {epoch+1}/{num_epochs} - Training")
        for batch_idx, (img, clin, label) in enumerate(tqdm(train_loader, desc="Training Batches")):
            img, clin, label = img.to(device), clin.to(device), label.to(device)
            optimizer.zero_grad()
            pred = model(img, clin)
            loss = focal_loss(pred, label)
            loss.backward()
            optimizer.step()

            batch_size = img.size(0)
            batch_loss = loss.item()
            batch_acc = accuracy(pred, label).item()
            batch_f1 = f1_score(pred, label).item()
            batch_auc = auroc(pred, label).item()

            train_loss += batch_loss * batch_size
            train_acc += batch_acc * batch_size
            train_f1 += batch_f1 * batch_size
            train_auc += batch_auc * batch_size
            train_samples += batch_size

        train_loss /= train_samples
        train_acc /= train_samples
        train_f1 /= train_samples
        train_auc /= train_samples

        model.eval()
        val_loss, val_acc, val_f1, val_auc = 0.0, 0.0, 0.0, 0.0
        val_samples = 0

        print(f"\nEpoch {epoch+1}/{num_epochs} - Validation")
        with torch.no_grad():
            for batch_idx, (img, clin, label) in enumerate(tqdm(val_loader, desc="Validation Batches")):
                img, clin, label = img.to(device), clin.to(device), label.to(device)
                pred = model(img, clin)
                loss = focal_loss(pred, label)

                batch_size = img.size(0)
                batch_loss = loss.item()
                batch_acc = accuracy(pred, label).item()
                batch_f1 = f1_score(pred, label).item()
                batch_auc = auroc(pred, label).item()

                val_loss += batch_loss * batch_size
                val_acc += batch_acc * batch_size
                val_f1 += batch_f1 * batch_size
                val_auc += batch_auc * batch_size
                val_samples += batch_size

        val_loss /= val_samples
        val_acc /= val_samples
        val_f1 /= val_samples
        val_auc /= val_samples

        print(f"\nEpoch {epoch+1} Summary:")
        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}, AUC: {train_auc:.4f}")
        print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}, AUC: {val_auc:.4f}")

        epoch_save_path = f"/content/drive/MyDrive/dl_package/epoch_{epoch+1}_multimodal_efficientnet.pth"
        torch.save(model.state_dict(), epoch_save_path)
        print(f"Saved model: {epoch_save_path}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "/content/drive/MyDrive/dl_package/best_multimodal_efficientnet.pth")
            print(f"New best model saved with Val F1: {best_f1:.4f}")
            patience_counter = 0
        else:
            patience_counter += 1

        scheduler.step(val_f1)
        print(f"Current learning rate: {optimizer.param_groups[0]['lr']:.6f}")

        if patience_counter >= patience:
            print("Early stopping triggered")
            break

    model.load_state_dict(best_model_wts)
    print("Loaded best model weights")
    return model

# Classification Report
def generate_classification_report(model, test_loader, device, weights_path="/content/drive/MyDrive/dl_package/best_multimodal_efficientnet.pth"):
    model.load_state_dict(torch.load(weights_path))
    model.to(device)
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for img, clin, label in test_loader:
            img, clin, label = img.to(device), clin.to(device), label.to(device)
            pred = model(img, clin)
            pred_classes = torch.argmax(pred, dim=1)
            all_preds.extend(pred_classes.cpu().numpy())
            all_labels.extend(label.cpu().numpy())

    report = classification_report(all_labels, all_preds, target_names=['AD', 'MCI', 'CN'], digits=4)
    print("EfficientNet Classification Report:")
    print(report)

    results_df = pd.DataFrame({
        'true_label': all_labels,
        'predicted_label': all_preds
    })
    results_df.to_csv("/content/drive/MyDrive/dl_package/test_predictions_efficientnet.csv", index=False)
    print("Saved predictions to /content/drive/MyDrive/dl_package/test_predictions_efficientnet.csv")

    return report, results_df

model = train_model(model, train_loader, val_loader, optimizer, num_epochs=50, patience=10)
report, results_df = generate_classification_report(model, test_loader, device)

Using device: cuda
Train samples: 725
Val samples: 242
Test samples: 242
Class weights: tensor([0.9986, 1.0028, 0.9986], device='cuda:0')

Epoch 1/50 - Training


Training Batches:  23%|██▎       | 21/91 [00:05<00:15,  4.49it/s]/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
Training Batches: 100%|██████████| 91/91 [00:14<00:00,  6.28it/s]



Epoch 1/50 - Validation


Validation Batches: 100%|██████████| 31/31 [00:02<00:00, 12.85it/s]



Epoch 1 Summary:
Train - Loss: 0.3135, Acc: 0.4041, F1: 0.3574, AUC: 0.5780
Val   - Loss: 0.3296, Acc: 0.3140, F1: 0.2238, AUC: 0.4330
Saved model: /content/drive/MyDrive/dl_package/epoch_1_multimodal_efficientnet.pth
New best model saved with Val F1: 0.2238
Current learning rate: 0.000100

Epoch 2/50 - Training


...


Epoch 44 Summary:
Train - Loss: 0.0185, Acc: 0.9766, F1: 0.9694, AUC: 0.9485
Val   - Loss: 0.3481, Acc: 0.8223, F1: 0.7936, AUC: 0.8640
Saved model: /content/drive/MyDrive/dl_package/epoch_44_multimodal_efficientnet.pth
Current learning rate: 0.000001

Epoch 45/50 - Training


Training Batches: 100%|██████████| 91/91 [00:25<00:00,  3.57it/s]



Epoch 45/50 - Validation


Validation Batches: 100%|██████████| 31/31 [00:07<00:00,  4.11it/s]



Epoch 45 Summary:
Train - Loss: 0.0056, Acc: 0.9917, F1: 0.9804, AUC: 0.9632
Val   - Loss: 0.3712, Acc: 0.8223, F1: 0.7913, AUC: 0.8686
Saved model: /content/drive/MyDrive/dl_package/epoch_45_multimodal_efficientnet.pth
Current learning rate: 0.000001
Early stopping triggered
Loaded best model weights
EfficientNet Classification Report:
              precision    recall  f1-score   support

          AD     0.7957    0.9250    0.8555        80
         MCI     0.8596    0.6049    0.7101        81
          CN     0.7283    0.8272    0.7746        81

    accuracy                         0.7851       242
   macro avg     0.7945    0.7857    0.7801       242
weighted avg     0.7945    0.7851    0.7798       242

Saved predictions to /content/drive/MyDrive/dl_package/test_predictions_efficientnet.csv
